In [26]:
# Import Necessary Libraries

from dotenv import load_dotenv, dotenv_values
import google.generativeai as genai
from IPython.display import Markdown, display
import os

# Load .env file

load_dotenv()

# Set the TESSDATA_PREFIX environment variable

tessdata = os.getenv("TESSDATA_PREFIX")
os.environ['TESSDATA_PREFIX'] = tessdata

# Set the Google api key

my_api_key = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=my_api_key)

In [37]:
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings,GoogleGenerativeAI

llm = GoogleGenerativeAI(model ="gemini-pro",temperature=0)
embedding = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004") 
model = ChatGoogleGenerativeAI(model= "gemini-1.5-flash", temperature = 0)

In [62]:
from typing import Literal

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

# Data model
class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""

    datasource: Literal["python", "js", "golang"] = Field(
        ...,
        description="Given a user question choose which datasource would be most relevant for answering their question",
    )

# LLM with function call 
structured_llm = model.with_structured_output(RouteQuery)

# Prompt 
system = """You are an expert at routing a user question to the appropriate data source.

Based on the programming language the question is referring to, route it to the relevant data source."""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

# Define router 
chain = prompt | structured_llm


In [63]:
from langchain_core.prompts import PromptTemplate

python_chain = PromptTemplate.from_template(
    """You are an expert in python. \
Respond to the following question:

Question: {question}
Answer:"""
) | ChatGoogleGenerativeAI(model= "gemini-1.5-flash", temperature = 0)
js_chain = PromptTemplate.from_template(
    """You are an expert in js. \
Respond to the following question:

Question: {question}
Answer:"""
) | ChatGoogleGenerativeAI(model= "gemini-1.5-flash", temperature = 0)
golang_chain = PromptTemplate.from_template(
    """You are an expert in golang. \
Respond to the following question:
Question: {question}
Answer:"""
) | ChatGoogleGenerativeAI(model= "gemini-1.5-flash", temperature = 0)

In [64]:
def choose_route(question):
    result = chain.invoke({"question": question})
    if "python" in result.datasource.lower():
        ### Logic here 
        print("Using Python")
        return python_chain
    elif "js" in result.datasource.lower():
        ### Logic here 
        print("Using js")
        return js_chain
    else:
        ### Logic here 
        print("Using Golang")
        return golang_chain

from langchain_core.runnables import RunnableLambda

#full_chain = router | RunnableLambda(choose_route)

In [70]:
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser

full_chain = {"topic": chain, "question": lambda x: x["question"]} | RunnableLambda(
    choose_route ) |StrOutputParser()

In [72]:
result = full_chain.invoke({"question": "How to work with Python in windows"})
print(result)

Using Python
## Working with Python in Windows: A Comprehensive Guide

Here's a breakdown of how to work with Python in Windows, covering installation, development environment setup, and essential tools:

**1. Installation:**

* **Download Python:** Head to the official Python website ([https://www.python.org/downloads/windows/](https://www.python.org/downloads/windows/)) and download the latest stable version. Choose the installer appropriate for your system (32-bit or 64-bit).
* **Run the Installer:** Double-click the downloaded file and follow the on-screen instructions. Ensure you check the "Add Python to PATH" option during installation. This makes it easier to run Python from the command line.
* **Verify Installation:** Open your command prompt (cmd) and type `python --version`. You should see the installed Python version.

**2. Development Environment Setup:**

* **Text Editor/IDE:** You can use a simple text editor like Notepad++ or Sublime Text for writing Python code. However